In [11]:
import numpy as np
import pandas as pd
import datetime as dt
import pathlib

import market_data as md
import alpha_vantage_api as av
import estimators
import performance_estimators as pe
import strategies as strat

# Read dataset
readFromFile: bool = True

if readFromFile:
    # CSV with SOFR
    sofr = md.load_sofr_dataset(file_path=r'SOFR.csv')
    stocks = md.load_spot_dataset(file_path=r'yahoo_data.csv')
    raw_data = stocks.merge(sofr, on='timestamp', how='inner')
else:
    # prices = pd.read_csv( r'prices.csv',
    #                       sep=';',
    #                       index_col='timestamp',
    #                       parse_dates = True)
    pass

etf_list = ['SPY', 'QQQ', 'SOXX', 'VOO', 'IWM', 'SQQQ', 'SOXS',
            'XLE', 'XLK', 'XLV', 'IVV', 'INDA', 'ARKK', 'EEM',
            'FXI', 'SXRT', 'TLT', 'LQD', 'IEF', 'SHY', 'VIXY']

comentarios

In [12]:
underlying = etf_list[4]
underlying = 'SPY'
fields = ['Adj Close']
init_time = dt.date(2010, 1, 1)

prices_df = md.get_historical_prices(prices=raw_data,
                                     underlying=underlying,
                                     fields=fields,
                                     funding_name='sofr')\
    .rename(columns={'Adj Close': 'close'})\
    .loc[lambda df: df.index.date > init_time]

# Define Parameters for trading strategy
sma_params = strat.SMA_Params( 
    execution_fee = 2.0 / 10_000,
    mean_slippage = 1.0/10_000,
    borrowing_cost = 3.0 / 100.0,
    bet_size = 10_000,  # in cash term,
    # vol_scaling_func = None,
    vol_scaling_func = lambda vol: np.min([1.0,0.15/vol]),
    short_sma_window = 10,
    long_sma_window = 50,
    feature_col = 'sma_long'
    )

# Define parameters for performance statistics
perf_params = {
    'var_pctl': 0.99,
    'risk_free': 0.0,
    'target_return': 0.0
}

In [13]:
prices_df

,close,sofr
timestamp,,
2010-01-04,87.129929,0.0000
2010-01-05,87.360603,0.0011
2010-01-06,87.422089,0.0009
2010-01-07,87.791115,0.0006
2010-01-08,88.083237,0.0004
...,...,...
2024-01-11,476.350006,0.0531
2024-01-12,476.679993,0.0531
2024-01-16,474.929993,0.0532
